# 05 · Model Cards （一页式模型摘要）🪪

该卡片从 `outputs/` 读取：metrics、阈值推荐、SHAP 图等，拼成一页式摘要（便于汇报/附录）。

In [1]:

# %% [bootstrap]
import sys, subprocess, importlib
for imp, spec in {"yaml":"pyyaml==6.0.2","joblib":"joblib==1.4.2"}.items():
    try: importlib.import_module(imp)
    except Exception:
        subprocess.run([sys.executable,"-m","pip","install",spec,"-q","--disable-pip-version-check","--no-input"], check=True)
import yaml, json, pandas as pd
from pathlib import Path
from IPython.display import display, Markdown, Image

PROJECT_ROOT = Path.cwd().resolve().parents[0] if Path.cwd().name == "notebooks" else Path.cwd()
CONF = yaml.safe_load((PROJECT_ROOT/"conf/config.yaml").read_text(encoding="utf-8"))
OUT_DIR = PROJECT_ROOT / "outputs"
TAB_DIR = PROJECT_ROOT / "outputs" / "tables"
FIG_DIR = PROJECT_ROOT / "outputs" / "figures"

MODEL  = CONF.get("models",{}).get("compare",["random_forest"])[0]
METHOD = "isotonic" if CONF.get("evaluation",{}).get("calibration", True) else "raw"

SUFFIX = "" if METHOD=="raw" else f"_{METHOD}"
p_metrics = OUT_DIR / f"metrics_test_{MODEL}{SUFFIX}.csv"
p_thrj    = OUT_DIR / f"threshold_scan_{MODEL}{SUFFIX}_summary.json"
p_shapbar = OUT_DIR / "shap" / f"shap_importance_bar_{MODEL}.png"
p_roc     = OUT_DIR / f"roc_test_{MODEL}{SUFFIX}.png"
p_calib   = OUT_DIR / f"calibration_test_{MODEL}{SUFFIX}.png"

md_lines = []
if p_metrics.exists():
    df = pd.read_csv(p_metrics)
    display(df)
    row = df.iloc[0].to_dict()
    md_lines += [
        f"**Model**: `{MODEL}`  |  **Method**: `{METHOD}`",
        f"- AUC: **{row.get('roc_auc', float('nan')):.3f}**  (95% CI {row.get('roc_auc_lo', float('nan')):.3f}–{row.get('roc_auc_hi', float('nan')):.3f})",
        f"- AP : **{row.get('ap', float('nan')):.3f}**  (95% CI {row.get('ap_lo', float('nan')):.3f}–{row.get('ap_hi', float('nan')):.3f})",
        f"- Brier: **{row.get('brier', float('nan')):.3f}**",
        f"- Acc/F1/Prec/Rec (thr=0.5): **{row.get('accuracy', float('nan')):.3f} / {row.get('f1', float('nan')):.3f} / {row.get('precision', float('nan')):.3f} / {row.get('recall', float('nan')):.3f}**",
    ]

thr_lines = []
if p_thrj.exists():
    js = json.loads(p_thrj.read_text(encoding="utf-8"))
    for k in ["youden","f1max","sens_target","spec_target"]:
        if k in js:
            thr = js[k].get("thr", None)
            desc = {"youden":"Youden J","f1max":"F1-max","sens_target":"sens≥target","spec_target":"spec≥target"}[k]
            if thr is not None:
                thr_lines.append(f"- {desc}: **thr={thr:.3f}**")
if thr_lines:
    md_lines.append("\n**Recommended thresholds**:\n" + "\n".join(thr_lines))

display(Markdown("\n\n".join(md_lines) if md_lines else "_No metrics found._"))

# 展示关键图
for p in [p_roc, p_calib, p_shapbar]:
    if p.exists():
        display(Image(filename=str(p)))


_No metrics found._

In [2]:

# %% [export card]
# 将卡片导出为 Markdown，便于提交/留档
card_md = PROJECT_ROOT / "outputs" / "tables" / f"model_card_{MODEL}{SUFFIX}.md"
card_md.parent.mkdir(parents=True, exist_ok=True)
card_md.write_text("See the notebook outputs; this placeholder ensures an artifact for tracking.\n", encoding="utf-8")
print("[ok] exported placeholder:", card_md)


[ok] exported placeholder: /public/home/aojiang/海南医科大学/icu-lymphoma-ml-repro/outputs/tables/model_card_logistic_isotonic.md
